## **Flights Data Exploration Challenge**

A significant part of a a data scientist's role is to explore, analyze, and visualize data. In this challenge, you'll explore a real-world dataset containing flights data from the US Department of Transportation.

Let's start by loading the required packages.


In [ ]:
# Load the packages in the tidyverse into the current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(summarytools)
  library(glue)
  library(patchwork)
  })


Now, we can import the into R and start doing some data science on it!



In [ ]:
# Load and view the data
df_flights <- read_csv("https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/challenges/data/flights.csv", show_col_types = FALSE)

df_flights %>% 
  slice_head(n = 7)


The dataset contains observations of US domestic flights in 2013, and consists of the following fields:

-   **Year**: The year of the flight (all records are from 2013)

-   **Month**: The month of the flight

-   **DayofMonth**: The day of the month on which the flight departed

-   **DayOfWeek**: The day of the week on which the flight departed - from 1 (Monday) to 7 (Sunday)

-   **Carrier**: The two-letter abbreviation for the airline.

-   **OriginAirportID**: A unique numeric identifier for the departure aiport

-   **OriginAirportName**: The full name of the departure airport

-   **OriginCity**: The departure airport city

-   **OriginState**: The departure airport state

-   **DestAirportID**: A unique numeric identifier for the destination aiport

-   **DestAirportName**: The full name of the destination airport

-   **DestCity**: The destination airport city

-   **DestState**: The destination airport state

-   **CRSDepTime**: The scheduled departure time

-   **DepDelay**: The number of minutes departure was delayed (flight that left ahead of schedule have a negative value)

-   **DelDelay15**: A binary indicator that departure was delayed by more than 15 minutes (and therefore considered "late")

-   **CRSArrTime**: The scheduled arrival time

-   **ArrDelay**: The number of minutes arrival was delayed (flight that arrived ahead of schedule have a negative value)

-   **ArrDelay15**: A binary indicator that arrival was delayed by more than 15 minutes (and therefore considered "late")

-   **Cancelled**: A binary indicator that the flight was cancelled

Your challenge is to explore the flight data to analyze possible factors that affect delays in departure or arrival of a flight.

1.  Start by cleaning the data.

    -   Identify any null or missing data, and impute appropriate replacement values.

    -   Identify and eliminate any outliers in the **DepDelay** and **ArrDelay** columns.

2.  Explore the cleaned data.

    -   View summary statistics for the numeric fields in the dataset.

    -   Determine the distribution of the **DepDelay** and **ArrDelay** columns.

    -   Use statistics, aggregate functions, and visualizations to answer the following questions:

        -   *What are the average (mean) departure and arrival delays?*

        -   *How do the carriers compare in terms of arrival delay performance?*

        -   *Is there a noticable difference in arrival delays for different days of the week?*

        -   *Which departure airport has the highest average departure delay?*

        -   *Do **late** departures tend to result in longer arrival delays than on-time departures?*

        -   *Which route (from origin airport to destination airport) has the most **late** arrivals?*

        -   *Which route has the highest average arrival delay?*

Sometimes, when we have a lot of columns in our data, it may difficult to get a grip of the data at first sight using `slice_head`

`glimpse` produces a transposed version where columns run down the page, and data runs across. This makes it possible to see every column in a data frame. Into the bargain, it also shows the dimension of the tibble and underlying data types of the columns.


In [ ]:
# Get a glimpse of your data
df_flights %>% 
  glimpse()


## Clean missing values

Once you have imported your data, it is always a good idea to clean it. Sadly, this is often chronically underestimated, yet it's a fundamental step required for the subsequent operations in data analysis.

Let's find how many null values there are for each column.


In [ ]:
# Find how many null values there are for each column.
colSums(is.na(df_flights))


Hmm, looks like there are some NA (missing values) `late departure` indicators. Departures are considered late if the delay is 15 minutes or more, so let's see the delays for the ones with an NA late indicator:



In [ ]:
# Filter the tibble to obtain rows where there is a missing value
# and then select DepDelay and DepDel15 columns
df_flights %>% 
  filter(rowSums(is.na(.)) > 0) %>% 
  select(DepDelay,DepDel15) %>% 
  slice_head(n = 20)


We can't see them all in this display, but from the first 20 observations, it looks like they may all have delay of 0. Let's check by looking at the summary statistics for these records:



In [ ]:
# Get summary statistics using summary function
df_flights %>% 
  filter(rowSums(is.na(.)) > 0) %>% 
  select(DepDelay) %>%
  summary()


The min, max, and mean are all 0; so it seems that none of these were actually *late* departures. Let's replace the missing values in **DepDel15** column with a 0 and confirm there are no more missing values.



In [ ]:
# Replace missing values in DepDel15 with 0
df_flights <- df_flights %>% 
  mutate(DepDel15 = replace_na(DepDel15, 0))

# Confirm there are no missing values
anyNA(df_flights)


Good job 👏! No missing values now. Let's take this a little further.

### **Clean outliers**

An outlier is a data point that differs significantly from other observations. Let's view the distribution and summary statistics for the **DepDelay** and **ArrDelay** columns.


In [ ]:
# Function to show summary stats and distribution for a column
show_distribution <- function(var_data, binwidth) {
  
  # Get summary statistics by first extracting values from the column
  min_val <- min(pull(var_data))
  max_val <- max(pull(var_data))
  mean_val <- mean(pull(var_data))
  med_val <- median(pull(var_data))
  mod_val <- statip::mfv(pull(var_data))

  # Print the stats
  stats <- glue::glue(
  'Minimum: {format(round(min_val, 2), nsmall = 2)}
   Mean: {format(round(mean_val, 2), nsmall = 2)}
   Median: {format(round(med_val, 2), nsmall = 2)}
   Mode: {format(round(mod_val, 2), nsmall = 2)}
   Maximum: {format(round(max_val, 2), nsmall = 2)}'
  )
  
  theme_set(theme_light())
  # Plot the histogram
  hist_gram <- ggplot(var_data) +
  geom_histogram(aes(x = pull(var_data)), binwidth = binwidth,
                 fill = "midnightblue", alpha = 0.7, boundary = 0.4) +
    
  # Add lines for the statistics
  geom_vline(xintercept = min_val, color = 'gray33', linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = mean_val, color = 'cyan', linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = med_val, color = 'red', linetype = "dashed", size = 1.3 ) +
  geom_vline(xintercept = mod_val, color = 'yellow', linetype = "dashed", size = 1.3 ) +
  geom_vline(xintercept = max_val, color = 'gray33', linetype = "dashed", size = 1.3 ) +
    
  # Add titles and labels
  ggtitle('Data Distribution') +
  xlab('')+
  ylab('Frequency') +
  theme(plot.title = element_text(hjust = 0.5))
  
  # Plot the box plot
  bx_plt <- ggplot(data = var_data) +
  geom_boxplot(mapping = aes(x = pull(var_data), y = 1),
               fill = "#E69F00", color = "gray23", alpha = 0.7) +
    
    # Add titles and labels
  xlab("Value") +
  ylab("") +
  theme(plot.title = element_text(hjust = 0.5))
  
  
  # To return multiple outputs, use a `list`
  return(
    
    list(stats,
         hist_gram / bx_plt)) # End of returned outputs
  
} # End of function


What's the distribution of **DepDelay**: The number of minutes departure was delayed?



In [ ]:
# Call the function
show_distribution(var_data = select(df_flights, DepDelay), binwidth = 100)


What's the distribution of **ArrDelay**: The number of minutes arrival was delayed?



In [ ]:
# Call the function
show_distribution(var_data = select(df_flights, ArrDelay), binwidth = 100)


There are a outliers at the lower and upper ends of both variables. Let's trim the data so that we include only rows where the values for these fields are within the 1st and 90th percentile.



In [ ]:
# Trim outliers for ArrDelay based on 1% and 90% percentiles
# Produce quantiles corresponding to 1% and 90%
ArrDelay_01pcntile <- df_flights %>% 
  pull(ArrDelay) %>% 
  quantile(probs = 1/100, names = FALSE)
ArrDelay_90pcntile <- df_flights %>% 
  pull(ArrDelay) %>% 
  quantile(probs = 90/100, names = FALSE)

# Filter data to remove outliers
df_flights <- df_flights %>% 
  filter(ArrDelay > ArrDelay_01pcntile, ArrDelay < ArrDelay_90pcntile)


Now, let's do the same for DepDelay column.



In [ ]:
# Trim outliers for DepDelay based on 1% and 90% percentiles
# Produce quantiles corresponding to 1% and 90%
DepDelay_01pcntile <- df_flights %>% 
  pull(DepDelay) %>% 
  quantile(probs = 1/100, names = FALSE)
DepDelay_90pcntile <- df_flights %>% 
  pull(DepDelay) %>% 
  quantile(probs = 90/100, names = FALSE)

# Filter data to remove outliers
df_flights <- df_flights %>% 
  filter(DepDelay > DepDelay_01pcntile, DepDelay < DepDelay_90pcntile)


What's the distribution of **DepDelay**: The number of minutes departure was delayed?



In [ ]:
# Call the function
show_distribution(var_data = select(df_flights, DepDelay), binwidth = 2)


What's the distribution of **ArrDelay**: The number of minutes arrival was delayed?



In [ ]:
# Call the function
show_distribution(var_data = select(df_flights, ArrDelay), binwidth = 2)


Much better 🤩!

Once the data is all cleaned up, we can now begin doing some exploratory analysis.

## **Explore the data**

Let's start with an overall view of the summary statistics for the numeric columns.


In [ ]:
# Obtain common summary statistics using summarytools package
df_flights %>% 
  descr(stats = "common")


### **What are the mean departure and arrival delays?**



In [ ]:
# Summarise the departure and arrival delays by finding the mean
df_flights %>% 
  summarise(across(contains("delay"), mean))


### **How do the carriers compare in terms of arrival delay performance?**



In [ ]:
df_flights %>% 
  ggplot() + 
  geom_boxplot(mapping = aes(x = Carrier, y = ArrDelay))


### **How do the carriers compare in terms of departure delay performance?**

We can also try and rearrange the `Carrier` levels in ascending order of the delay time and sprinkle some color to the plots too.


In [ ]:
df_flights %>% 
  mutate(Carrier = fct_reorder(Carrier, DepDelay)) %>% 
  ggplot() + 
  geom_boxplot(mapping = aes(x = Carrier, y = DepDelay, color = Carrier), show.legend = FALSE)


Alternatively, to create the above plots, we can use `purr::map()` which allows us to apply a function to each column. See `?map` for more details.



In [ ]:
map(df_flights %>% select(ArrDelay, DepDelay), ~ ggplot(df_flights) + 
  geom_boxplot(mapping = aes(x = Carrier, y = .x)) + ylab(""))


### **Are some days of the week more prone to arrival delays than others?**



In [ ]:
# Encode day of the week as a categorical and make boxplots
df_flights %>% 
  mutate(DayOfWeek = factor(DayOfWeek)) %>% 
  ggplot() + 
  geom_boxplot(mapping = aes(x = DayOfWeek, y = ArrDelay), show.legend = FALSE)


### **Are some days of the week more prone to departure delays than others?**



In [ ]:
# Encode day of the week as a categorical variable and make boxplots
df_flights %>% 
  mutate(DayOfWeek = factor(DayOfWeek)) %>% 
  ggplot() + 
  geom_boxplot(mapping = aes(x = DayOfWeek, y = DepDelay, color = DayOfWeek), show.legend = FALSE) +
  scale_color_brewer(palette = "Dark2")


### **Which departure airport has the highest average departure delay?**



In [ ]:
# Use group_by %>% summarize to find airports with highest avg DepDelay
mean_departure_delays <- df_flights %>% 
  group_by(OriginAirportName) %>% 
  summarise(mean_dep_delay_time = mean(DepDelay)) %>%
  arrange(desc(mean_dep_delay_time))

mean_departure_delays %>% 
  slice_head(n = 7)


Fantastic!

Let's represent this using bar plots.


In [ ]:
mean_departure_delays %>%
  # Sort factor levels in descending order of delay time
  mutate(OriginAirportName = fct_reorder(OriginAirportName, desc(mean_dep_delay_time))) %>% 
  ggplot() +
  geom_col(mapping = aes(x = OriginAirportName, y = mean_dep_delay_time), fill = "midnightblue", alpha = 0.7) +
  theme(
    # Rotate X markers so we can read them
    axis.text.x = element_text(angle = 90)
  )


Could you try and guess why Chicago Airport has most departure delay time or why Long Beach has the least?

### **Do late departures tend to result in longer arrival delays than on-time departures?**


In [ ]:
df_flights %>%
  mutate(DepDel15 = factor(DepDel15)) %>% 
  ggplot() +
  geom_boxplot(mapping = aes(x = DepDel15, y = ArrDelay, fill = DepDel15)) 


Does this surprise you 🤷?

### **Which route (from origin airport to destination airport) has the most late arrivals?**


In [ ]:
# Add a "Route" column
df_flights <- df_flights %>% 
  mutate(Route = paste(OriginAirportName, DestAirportName, sep = ">")) 

# Make grouped summaries to find the total delay associated with a particular airport
df_flights %>% 
  group_by(Route) %>% 
  summarise(ArrDel15 = sum(ArrDel15)) %>% 
  arrange(desc(ArrDel15)) 


### **Which route has the highest average arrival delay?**



In [ ]:
# Create grouped summaries of the arrival delay time
df_flights %>% 
  group_by(Route) %>% 
  summarise(ArrDelay = mean(ArrDelay)) %>% 
  arrange(desc(ArrDelay)) 


Congratulations on finishing the first challenge 🎉! We'll wrap it at that for now. Of course there are other ways to approach this challenge. So please feel free to experiment, google and share your solutions with friends.

See you in the next module where we get started with Machine Learning!

Happy Learning,

[Eric,](https://twitter.com/ericntay) Gold Microsoft Learn Student Ambassador.
